In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.models import model_from_json

In [3]:
labels_t = pd.read_csv("../input/datathonindoml-2022/train_labels.csv")

In [4]:
from typing import Tuple

def resize_with_pad(image: np.array, 
                    new_shape: Tuple[int, int], 
                    padding_color: Tuple[int] = (255, 255, 255)) -> np.array:
    original_shape = (image.shape[1], image.shape[0])
    ratio = float(max(new_shape))/max(original_shape)
    new_size = tuple([int(x*ratio) for x in original_shape])
    image = cv2.resize(image, new_size)
    delta_w = new_shape[0] - new_size[0]
    delta_h = new_shape[1] - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=padding_color)
    return image

In [5]:
def test(model_path,data_path,weight_path):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(weight_path)
    loaded_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0005),metrics=['accuracy'])
    X_val=[]
    y_val=[]
    path = data_path
    for f in os.listdir(path):
        y_val.append(os.path.splitext(f)[0])
        temp2 = cv2.imread(path+f)
        temp2 = cv2.cvtColor(temp2,cv2.COLOR_BGR2YUV)
        temp2[:,:,0] = cv2.equalizeHist(temp2[:,:,0])
        temp2 = cv2.cvtColor(temp2, cv2.COLOR_YUV2BGR)
        temp2 = resize_with_pad(temp2, (256,256))
        temp2 = cv2.cvtColor(temp2, cv2.COLOR_BGR2GRAY)
        temp2 = np.expand_dims(temp2,axis=2)
        X_val.append(temp2)
    X_val = np.array(X_val)
    X_val = X_val.astype('float32')
    X_val/=255
    model=loaded_model
    y_pred=model.predict(X_val)
    prediction=[np.argmax(element) for element in y_pred]
    save = pd.DataFrame({'id':y_val,'label':prediction})
    save.to_csv('submission.csv',index=False)

In [6]:
test('../input/models/model.json',"../input/datathonindoml-2022/validation/validation/","../input/models/weights.best.hdf5")

2022-10-09 09:39:35.054058: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 09:39:35.163686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 09:39:35.164501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 09:39:35.166324: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil